In [2]:
from astropy import cosmology
from astropy.cosmology import Planck15, FlatLambdaCDM, z_at_value
from astropy.constants import *
import scipy.integrate as integrate
from scipy.interpolate import UnivariateSpline as spl
from scipy.constants import *
import astropy.units as uni
import numpy as np
import h5py
import math
import time,sys,os
import datetime

In [3]:
# Builds a DM lighcone from Gadget dark matter snapshots

In [4]:
print ('All imports OK')

All imports OK


In [5]:
# initialise cosmology
cosmo = FlatLambdaCDM(H0=100, Om0=0.30711520) 

In [6]:
# Calculate z for range of comoving distances
z = np.zeros(3000)
d_c = np.zeros(3000)
H = np.zeros(3000)

for d in range(1,3000):
    d_c[d] = d
    z[d] = z_at_value(cosmo.comoving_distance, d * uni.Mpc)
    H[d]  = cosmo.H(z[d]).value
    
# create spine for quick lookup
d2z = spl(d_c,z)
z2H = spl(z, H, s=0)

In [7]:
def read_header(snap):
    snapshot_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)
    if os.path.exists(snapshot_fname):
        with h5py.File(snapshot_fname,'r') as f:
            h = f['Header']
            z=h.attrs['Redshift']
    else:
            z = -1
    return z

In [8]:
first_snap = 110
snaps = 18
last_snap = first_snap + snaps

In [9]:
# read headers for all snapshots to get redshifts
snap_list = []
zz = np.empty(0, dtype = np.float32)
for snap in range(first_snap, last_snap):
    z = read_header(snap)
    if (z>=0):
        zz = np.append(zz,z)
        snap_list.append(snap)

snapshots = np.array(snap_list)
print('Valid snapshots =', len(snapshots))

# calculate maximum redshift for each snapshot bin boundary
# as mid point between nominal redshift value for snapshot
z_max = zz[:-1:] + (zz[1::] - zz[:-1:])/2
z_max = np.append(z_max, 2*zz[-1] - z_max[-1])

Valid snapshots = 14


In [10]:
snapshots

array([110, 111, 113, 114, 115, 116, 117, 119, 120, 121, 123, 124, 126,
       127])

In [11]:
zz

array([2.01266093e-01, 1.83048686e-01, 1.65107548e-01, 1.47438491e-01,
       1.30037388e-01, 1.12900176e-01, 9.60228526e-02, 7.94014768e-02,
       6.30321670e-02, 4.69111006e-02, 3.10345130e-02, 1.53986966e-02,
       2.22044605e-16, 2.22044605e-16])

In [12]:
# Calculate maximum comoving distance corresponding to redshift bin maxima
Dc_max = np.zeros(len(snapshots))
Dc_z = np.zeros(len(snapshots))
for sn in snapshots:
    ind = np.where(snapshots == sn)[0][0]
    Dc_max[ind] = cosmo.comoving_distance(z_max[ind]).value
    Dc_z[ind] = cosmo.comoving_distance(zz[ind]).value
Dc_min = 0   

In [13]:
Dc_z

array([ 5.74536988e+02,  5.24953935e+02,  4.75646728e+02,  4.26626360e+02,
        3.77903517e+02,  3.29488555e+02,  2.81391494e+02,  2.33621996e+02,
        1.86189359e+02,  1.39102503e+02,  9.23699644e+01,  4.59998830e+01,
       -2.09771740e-12, -2.09771740e-12])

In [15]:
# define particle datatype
vect = np.dtype([('x', np.float32),('y', np.float32),('z', np.float32)])
part = np.dtype([('pos', vect),('vel', vect),('ID', np.ulonglong), ('z', np.float32), ('r', np.float32),('RA', np.float32),('Dec', np.float32)])

In [18]:
# Load each snapshot
def do_snap(snap):  
    snapshot_fname = '/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/ordered_snapshot.snap_{0:03d}.hdf5'.format(snap)
    print('Loading snapshot {0:0d}, time = {1:%H:%M:%S}'.format(snap, datetime.datetime.now()))
    with h5py.File(snapshot_fname,'r') as f:
        pts = f['PartType1']
        
        for oct in range(reps): 
            #relocate origin to each corner of the simulation box for each octant
            
            if (reps>1):
                orig_x = oct%2 * BoxSize
                orig_y = oct//2%2 * BoxSize
                orig_z = oct//4 * BoxSize
            else:
                # for single box, set origin to mid point in box
                orig_x = BoxSize/2
                orig_y = BoxSize/2
                orig_z = BoxSize/2
    
            # translate particle positions to new origin for each octant
            x = pts['pos']['x'] - orig_x
            y = pts['pos']['y'] - orig_y
            z = pts['pos']['z'] - orig_z
            
            #repackage pos as vect
            pos_o = np.array(list(zip(x,y,z)), dtype = vect)
            
            # calculate comoving radial distance, RA and Dec
            r = np.sqrt(x*x + y*y + z*z)
            dec = np.rad2deg(np.arcsin(z/r))
            ra = np.rad2deg(np.arctan2(y,x))

            r_max = np.amax(r)
            r_min = np.amin(r)
            
            # lookup redshift corresponding to this r
            zz = d2z(r)           
            
            #Check whether particle within shell max and min
            ind = np.where(snapshots == snap)[0][0]
            max_r = Dc_max[ind]         
            if (ind >= Dc_max.size-1):
                min_r = 0
            else:
                min_r = Dc_max[ind+1]
                
            f = (r < max_r) & (r >= min_r)
            
            print('Snapshot {0:03d}, index = {5:03d}, r_max = {1:04.1f}, r_min = {2:04.1f}, Dc_max A = {3:04.1f}, Dc_max B = {4:04.1f}'
                  .format(snap, r_max, r_min, max_r, min_r, ind))
            
            print('Number of particles in snapshot {0:0d} = {1:0d}'.format(snap,f.sum()))

            # resize and append to dataset. Use f to filter only those galaxies within snapshot redshift boundaries
            #p = np.array(list(zip(pos_o[f], pts['vel'][f], pts['ID'][f], zz[f], r[f], ra[f], dec[f])),  dtype= part)
            p = np.array(list(zip(pos_o, pts['vel'], pts['ID'], zz, r, ra, dec)),  dtype= part)
            with h5py.File(fname, 'a') as fl:
                dsname = 'LC_Particles_snap_{0:03}'.format(snap)
                print ('Saving particles in dataset {0:s}'.format(dsname))
                parts= fl.create_dataset(dsname, data = p[f])

In [19]:
# open output file
lightcone_path = '/cosma7/data/dp004/dc-boot5/Lightcone/DM_shells_indexed/'
fname = lightcone_path + 'L750.hdf5'
reps = 1
BoxSize = 750.0

fo = h5py.File(fname,'w')
fo.close()

for sn in snapshots:
    do_snap(sn)

Loading snapshot 110, time = 10:57:45
Snapshot 110, index = 000, r_max = 649.5, r_min = 01.6, Dc_max A = 549.8, Dc_max B = 500.4
Number of particles in snapshot 110 = 8089358
Saving particles in dataset LC_Particles_snap_110
Loading snapshot 111, time = 11:07:23
Snapshot 111, index = 001, r_max = 649.5, r_min = 01.6, Dc_max A = 500.4, Dc_max B = 451.2
Number of particles in snapshot 111 = 16215875
Saving particles in dataset LC_Particles_snap_111
Loading snapshot 113, time = 11:16:15
Snapshot 113, index = 002, r_max = 649.5, r_min = 01.6, Dc_max A = 451.2, Dc_max B = 402.3
Number of particles in snapshot 113 = 22963114
Saving particles in dataset LC_Particles_snap_113
Loading snapshot 114, time = 11:25:20
Snapshot 114, index = 003, r_max = 649.5, r_min = 01.6, Dc_max A = 402.3, Dc_max B = 353.7
Number of particles in snapshot 114 = 26034590
Saving particles in dataset LC_Particles_snap_114
Loading snapshot 115, time = 11:33:34
Snapshot 115, index = 004, r_max = 649.5, r_min = 01.6, Dc_